# RAG 평가 파이프라인 (Full)

이 노트북은 `gold_evidence.csv`, `gold_fields.jsonl`, `questions.csv`를 이용해
K값/청크 전략을 교차 비교하고 Retrieval 및 Generation 지표를 계산합니다.


## 0) 설치 & 환경 변수

필요 패키지: `openai`, `pandas`, `numpy`, `pdfplumber`, `faiss-cpu`(권장), `rapidfuzz`, `tqdm`, `python-dotenv`


In [1]:
from __future__ import annotations

import os
import re
import json
from dataclasses import dataclass
from pathlib import Path
from typing import Optional

import numpy as np
import pandas as pd
import pdfplumber
from tqdm.auto import tqdm
from rapidfuzz import fuzz

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI()

EMBED_MODEL = "text-embedding-3-small"
GEN_MODEL_MAIN = "gpt-5-mini"
GEN_MODEL_AUX  = "gpt-5-nano"

/home/ohs3201/codeit/codeit-part3-team4/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1) 경로 설정

데이터 위치를 확인합니다.


In [2]:
BASE_DIR = Path.cwd().parent
if not BASE_DIR.exists():
    BASE_DIR = Path.cwd().parent

RAW_FILES_DIR = BASE_DIR / "data/raw/files"
EVAL_DIR = BASE_DIR / "data/raw/eval"

QUESTIONS_CSV = EVAL_DIR / "questions.csv"
GOLD_EVIDENCE_CSV = EVAL_DIR / "gold_evidence.csv"
GOLD_FIELDS_JSONL = EVAL_DIR / "gold_fields.jsonl"

print("BASE_DIR:", BASE_DIR)
print("RAW_FILES_DIR:", RAW_FILES_DIR, "exists:", RAW_FILES_DIR.exists())
print("EVAL_DIR:", EVAL_DIR, "exists:", EVAL_DIR.exists())

BASE_DIR: /home/ohs3201/codeit/codeit-part3-team4
RAW_FILES_DIR: /home/ohs3201/codeit/codeit-part3-team4/data/raw/files exists: True
EVAL_DIR: /home/ohs3201/codeit/codeit-part3-team4/data/raw/eval exists: True


## 2) 평가 데이터 로드

`questions.csv`, `gold_evidence.csv`, `gold_fields.jsonl`을 읽습니다.


In [3]:
def load_questions_csv(path: Path) -> pd.DataFrame:
    # columns: instance_id,qid,doc_id,question,type
    # question에 쉼표가 들어가도 파싱되도록, 앞 3개 + 마지막 1개 고정
    rows = []
    with path.open("r", encoding="utf-8", errors="replace") as f:
        header = f.readline().strip().split(",")
        assert header[:5] == ["instance_id","qid","doc_id","question","type"], header
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split(",")
            if len(parts) < 5:
                continue
            instance_id, qid, doc_id = parts[0], parts[1], parts[2]
            qtype = parts[-1]
            question = ",".join(parts[3:-1]).strip()
            rows.append({"instance_id": instance_id, "qid": qid, "doc_id": doc_id, "question": question, "type": qtype})
    return pd.DataFrame(rows)

def load_gold_fields_jsonl(path: Path) -> pd.DataFrame:
    rows = []
    with path.open("r", encoding="utf-8", errors="replace") as f:
        for line in f:
            if not line.strip():
                continue
            rows.append(json.loads(line))
    out = []
    for r in rows:
        iid = r["instance_id"]
        doc_id = r.get("doc_id", "")
        fields = r.get("fields", {}) or {}
        for k, v in fields.items():
            out.append({"instance_id": iid, "doc_id": doc_id, "field": k, "gold": v})
    return pd.DataFrame(out)

questions_df = load_questions_csv(QUESTIONS_CSV)
gold_evidence_df = pd.read_csv(GOLD_EVIDENCE_CSV)
gold_fields_df = load_gold_fields_jsonl(GOLD_FIELDS_JSONL)

display(questions_df.head(3))
display(gold_evidence_df.head(3))
display(gold_fields_df.head(3))

,instance_id,qid,doc_id,question,type
0,G_Q001,Q001,*,사업(용역)명은 무엇인가?,project_name
1,G_Q002,Q002,*,발주 기관(수요기관)은 어디인가?,agency
2,G_Q003,Q003,*,사업 목적(추진 배경)은 무엇인가?,purpose


,instance_id,doc_id,page_start,page_end,anchor_text
0,G_Q001,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,4.0,4.0,사업개요
1,G_Q002,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,1.0,1.0,벤처기업확인기관
2,G_Q003,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,3.0,3.0,추진배경 및 방향


,instance_id,doc_id,field,gold
0,G_Q001,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,project_name,벤처확인종합관리시스템 기능 고도화
1,G_Q002,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,agency,벤처기업확인기관
2,G_Q003,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,purpose,"[복수의결권주식, 스톡옵션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능..."


## 3) 평가 대상 문서 선택

특정 문서만 테스트하거나 전체 문서를 선택할 수 있습니다.


In [4]:
TARGET_DOCS = [
    "(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf",
]
GLOB_PATTERN: Optional[str] = None  # 예: "*벤처확인*.pdf"

def list_pdfs(folder: Path) -> list[Path]:
    return sorted(folder.glob("*.pdf"))

all_pdfs = list_pdfs(RAW_FILES_DIR)
print("PDF count:", len(all_pdfs))

if GLOB_PATTERN:
    DOC_PATHS = [p for p in all_pdfs if p.match(GLOB_PATTERN)]
else:
    DOC_PATHS = [p for p in all_pdfs if p.name in set(TARGET_DOCS)]

if not DOC_PATHS:
    print("⚠️ 선택된 문서가 없어서 전체 문서를 대상으로 합니다.")
    DOC_PATHS = all_pdfs

for p in DOC_PATHS:
    print(" -", p.name)

PDF count: 100
 - (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf


## 4) PDF 로딩 + 청크 메타

`pdfplumber`로 텍스트를 추출하고 페이지 정보를 유지합니다.


In [ ]:
@dataclass
class Chunk:
    doc_id: str
    chunk_id: str
    text: str
    page_start: int
    page_end: int

def extract_pages(pdf_path: Path) -> list[dict]:
    pages = []
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            pages.append({"page_no": i + 1, "text": page.extract_text() or ""})
    return pages

## 5) 청크 전략

페이지 단위, 고정 길이, 문단 기반 등 다양한 전략을 비교합니다.


In [6]:
def chunk_page(pages: list[dict], doc_id: str) -> list[Chunk]:
    out = []
    for p in pages:
        txt = (p["text"] or "").strip()
        if txt:
            out.append(Chunk(doc_id, f"p{p['page_no']}", txt, p["page_no"], p["page_no"]))
    return out

def chunk_fixed_chars(pages: list[dict], doc_id: str, size: int = 1200, overlap: int = 200) -> list[Chunk]:
    full = []
    spans = []
    cur = 0
    for p in pages:
        t = (p["text"] or "") + ""
        a = cur
        full.append(t)
        cur += len(t)
        b = cur
        spans.append((a, b, p["page_no"]))
    full_text = "".join(full)

    def span_to_pages(s: int, e: int) -> tuple[int, int]:
        ps = [pg for (a, b, pg) in spans if not (b <= s or a >= e)]
        return (min(ps), max(ps)) if ps else (1, 1)

    out = []
    step = max(1, size - overlap)
    for i, s in enumerate(range(0, len(full_text), step)):
        e = min(len(full_text), s + size)
        txt = full_text[s:e].strip()
        if not txt:
            continue
        ps, pe = span_to_pages(s, e)
        out.append(Chunk(doc_id, f"c{i}", txt, ps, pe))
        if e >= len(full_text):
            break
    return out

def chunk_page_paragraph(pages: list[dict], doc_id: str, max_chars: int = 1500, min_chars: int = 300) -> list[Chunk]:
    out = []
    cid = 0
    for p in pages:
        page_no = p["page_no"]
        raw = (p["text"] or "").strip()
        if not raw:
            continue
        paras = [x.strip() for x in re.split(r"\s*+", raw) if x.strip()]
        buf = ""
        buf_start = page_no
        for para in paras:
            if len(buf) + len(para) + 2 <= max_chars:
                buf = (buf + "" + para).strip()
            else:
                if len(buf) >= min_chars:
                    out.append(Chunk(doc_id, f"pp{cid}", buf, buf_start, page_no))
                    cid += 1
                    buf = para
                    buf_start = page_no
                else:
                    buf = (buf + "" + para).strip()
        if buf.strip():
            out.append(Chunk(doc_id, f"pp{cid}", buf.strip(), buf_start, page_no))
            cid += 1
    return out

CHUNKERS = {
    "page": lambda pages, doc_id, **kw: chunk_page(pages, doc_id),
    "fixed_chars": lambda pages, doc_id, **kw: chunk_fixed_chars(pages, doc_id, size=kw.get("size", 1200), overlap=kw.get("overlap", 200)),
    "page_paragraph": lambda pages, doc_id, **kw: chunk_page_paragraph(pages, doc_id, max_chars=kw.get("max_chars", 1500), min_chars=kw.get("min_chars", 300)),
}

## 6) 임베딩 + 벡터 검색(FAISS 우선)

FAISS가 없으면 sklearn fallback을 사용합니다.


In [7]:
try:
    import faiss
    FAISS_AVAILABLE = True
except Exception:
    FAISS_AVAILABLE = False
    from sklearn.neighbors import NearestNeighbors


def embed_texts(texts: list[str], model: str = EMBED_MODEL, batch_size: int = 64) -> np.ndarray:
    vecs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
        batch = texts[i:i + batch_size]
        resp = client.embeddings.create(model=model, input=batch)
        vecs.extend([d.embedding for d in resp.data])
    arr = np.array(vecs, dtype=np.float32)
    return arr / (np.linalg.norm(arr, axis=1, keepdims=True) + 1e-12)


def build_index(embs: np.ndarray):
    if FAISS_AVAILABLE:
        index = faiss.IndexFlatIP(embs.shape[1])
        index.add(embs)
        return ("faiss", index)
    else:
        nn = NearestNeighbors(metric="cosine")
        nn.fit(embs)
        return ("sklearn", nn)


def search(index_pack, qv: np.ndarray, top_k: int = 10):
    kind, index = index_pack
    if kind == "faiss":
        scores, I = index.search(qv.reshape(1, -1).astype(np.float32), top_k)
        return scores[0], I[0]
    else:
        dists, I = index.kneighbors(qv.reshape(1, -1), n_neighbors=top_k)
        scores = 1.0 - dists[0]
        return scores, I[0]


def embed_query(q: str, model: str = EMBED_MODEL) -> np.ndarray:
    resp = client.embeddings.create(model=model, input=[q])
    v = np.array(resp.data[0].embedding, dtype=np.float32)
    return v / (np.linalg.norm(v) + 1e-12)

## 7) Retrieval 평가 (Recall@K, MRR@K)

`gold_evidence.csv`의 페이지 범위와 겹치는지로 평가합니다.


In [8]:
def overlaps(a_start: int, a_end: int, b_start: int, b_end: int) -> bool:
    return not (a_end < b_start or b_end < a_start)

def build_gold_evidence_map(df: pd.DataFrame) -> dict[str, list[tuple[int, int]]]:
    m: dict[str, list[tuple[int, int]]] = {}
    for _, r in df.iterrows():
        iid = str(r["instance_id"])
        ps = r.get("page_start")
        pe = r.get("page_end")
        if pd.isna(ps) or pd.isna(pe):
            # Skip evidence rows without page ranges (e.g., NOT_FOUND)
            continue
        m.setdefault(iid, []).append((int(ps), int(pe)))
    return m

GOLD_EVIDENCE = build_gold_evidence_map(gold_evidence_df)

def eval_retrieval(chunks: list[Chunk], index_pack, question: str, instance_id: str, k: int) -> dict[str, float]:
    qv = embed_query(question)
    _, idxs = search(index_pack, qv, top_k=k)
    gold_ranges = GOLD_EVIDENCE.get(instance_id, [])
    hit_rank = None
    for rank, ci in enumerate(idxs, start=1):
        if 0 <= int(ci) < len(chunks):
            c = chunks[int(ci)]
            if any(overlaps(c.page_start, c.page_end, ps, pe) for ps, pe in gold_ranges):
                hit_rank = rank
                break
    return {"recall": 1.0 if hit_rank else 0.0, "mrr": (1.0 / hit_rank) if hit_rank else 0.0}

## 8) Generation + 평가

검색된 청크로 컨텍스트를 만들고 답변을 생성합니다.


In [ ]:
GEN_SYSTEM = "너는 공공 RFP 문서에서 질문에 대한 '짧은 답'만 뽑는 분석가다. 추측 금지."
GEN_TEMPLATE = """[문서 발췌]
{context}

[질문]
{question}

[출력 규칙]
- 가능한 한 짧고 단정적인 답 1줄
- 문서에 없으면: NOT_FOUND
- 불필요한 설명 금지
"""


def build_context(chunks: list[Chunk], idxs: list[int], max_chars: int = 2000) -> str:
    parts, total = [], 0
    for ci in idxs:
        if not (0 <= int(ci) < len(chunks)):
            continue
        c = chunks[int(ci)]
        s = f"[p{c.page_start}~p{c.page_end} | {c.chunk_id}]\\n{c.text}".strip()
        if total + len(s) + 2 > max_chars:
            break
        parts.append(s)
        total += len(s) + 2
    return "\\n\\n".join(parts)


def generate_answer(question: str, context: str, model: str = GEN_MODEL_MAIN) -> str:
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": GEN_SYSTEM},
            {"role": "user", "content": GEN_TEMPLATE.format(context=context, question=question)},
        ],
        max_completion_tokens=512,
        reasoning_effort="low",
        verbosity="low",
    )
    text = (resp.choices[0].message.content or "").strip()
    return text.splitlines()[0].strip() if text else ""

AUX_PROMPT = """아래 답변에서 '최종 답'만 1줄로 정리해줘.
- 괄호/인용부호 제거
- 앞뒤 공백 제거
- 답이 없으면 그대로 'NOT_FOUND'

답변:
{answer}
"""


def normalize_with_nano(answer: str) -> str:
    resp = client.chat.completions.create(
        model=GEN_MODEL_AUX,
        messages=[
            {"role":"system","content":"텍스트 정규화 도우미."},
            {"role":"user","content": AUX_PROMPT.format(answer=answer)},
        ],
        max_completion_tokens=64,
    )
    text = (resp.choices[0].message.content or "").strip()
    return text.splitlines()[0].strip() if text else ""


def eval_gen(pred: str, gold: Optional[str], threshold: int = 80) -> dict[str, float]:
    pred = (pred or "").strip()
    fill = 1.0 if pred and pred.lower() not in {"", "없음"} else 0.0
    if gold is None or str(gold).strip() == "":
        return {"fill": fill, "match": np.nan, "sim": np.nan}
    gold = str(gold).strip()
    sim = fuzz.token_set_ratio(pred, gold)
    return {"fill": fill, "match": 1.0 if sim >= threshold else 0.0, "sim": float(sim)}

## 9) 실험 그리드

청크 전략과 K값을 교차 비교합니다.


In [10]:
CHUNK_GRID = [
    {"name": "page", "params": {}},
    {"name": "fixed_chars", "params": {"size": 1200, "overlap": 200}},
    {"name": "fixed_chars", "params": {"size": 2000, "overlap": 300}},
    {"name": "page_paragraph", "params": {"max_chars": 1500, "min_chars": 300}},
    {"name": "page_paragraph", "params": {"max_chars": 2500, "min_chars": 400}},
]
K_LIST = [3, 5, 10, 15]
SIM_THRESHOLD = 80
USE_NANO_NORMALIZER = False

## 10) 실행

실행 결과를 표로 정리하고 CSV로 저장합니다.


In [11]:
def get_questions_for_doc(doc_name: str) -> pd.DataFrame:
    m = (questions_df["doc_id"].astype(str) == doc_name) | (questions_df["doc_id"].astype(str) == "*")
    return questions_df[m].copy()

def get_gold_map_for_doc(doc_name: str) -> dict[tuple[str, str], str]:
    df = gold_fields_df[gold_fields_df["doc_id"].astype(str) == doc_name]
    return {(r["instance_id"], r["field"]): r["gold"] for _, r in df.iterrows()}

DEBUG_GEN = True
DEBUG_GEN_MAX = 3

def run_grid_for_doc(doc_path: Path) -> pd.DataFrame:
    doc_name = doc_path.name
    qdf = get_questions_for_doc(doc_name)
    if len(qdf) == 0:
        print("⚠️ no questions:", doc_name)
        return pd.DataFrame()

    gold_map = get_gold_map_for_doc(doc_name)
    pages = extract_pages(doc_path)

    rows = []
    for cfg in CHUNK_GRID:
        chunks = CHUNKERS[cfg["name"]](pages, doc_name, **cfg["params"])
        texts = [c.text for c in chunks]
        if not texts:
            continue
        embs = embed_texts(texts)
        index_pack = build_index(embs)

        for k in K_LIST:
            r_list, g_list = [], []
            for _, qr in tqdm(qdf.iterrows(), total=len(qdf), desc=f"{doc_name} | {cfg['name']} | k={k}"):
                iid = str(qr["instance_id"])
                question = str(qr["question"])
                qtype = str(qr["type"])

                # retrieval
                r = eval_retrieval(chunks, index_pack, question, iid, k)
                r_list.append(r)

                # generation (gold 있을 때만)
                gold = gold_map.get((iid, qtype))
                if gold is not None:
                    qv = embed_query(question)
                    _, idxs = search(index_pack, qv, top_k=k)
                    idxs = [int(x) for x in idxs if int(x) >= 0]
                    ctx = build_context(chunks, idxs)
                    pred = generate_answer(question, ctx)
                    if DEBUG_GEN and len(g_list) < DEBUG_GEN_MAX:
                        print("Q:", question)
                        print("GOLD:", gold)
                        print("PRED:", pred)
                        print("-"*40)
                    if USE_NANO_NORMALIZER:
                        pred = normalize_with_nano(pred)
                        if DEBUG_GEN and len(g_list) < DEBUG_GEN_MAX:
                            print("PRED_AFTER:", pred)
                            print("="*40)
                    g_list.append(eval_gen(pred, gold, threshold=SIM_THRESHOLD))
                else:
                    g_list.append({"fill": np.nan, "match": np.nan, "sim": np.nan})

            rows.append({
                "doc_id": doc_name,
                "chunk_strategy": cfg["name"],
                "chunk_params": json.dumps(cfg["params"], ensure_ascii=False),
                "k": k,
                "n_questions_total": len(qdf),
                "n_questions_with_evidence": int(sum(1 for x in qdf["instance_id"] if str(x) in set(gold_evidence_df["instance_id"].astype(str)))) ,
                "retrieval_recall@k": float(np.mean([x["recall"] for x in r_list])),
                "retrieval_mrr@k": float(np.mean([x["mrr"] for x in r_list])),
                "n_questions_with_gold_fields": int(sum(1 for x in qdf["instance_id"] if (str(x), str(qdf[qdf["instance_id"]==x]["type"].iloc[0])) in set(gold_map.keys()))),
                "gen_fill_rate": float(np.nanmean([x["fill"] for x in g_list])),
                "gen_match_rate": float(np.nanmean([x["match"] for x in g_list])),
                "gen_avg_similarity": float(np.nanmean([x["sim"] for x in g_list])),
            })
    return pd.DataFrame(rows)

all_results = []
for doc_path in DOC_PATHS:
    all_results.append(run_grid_for_doc(doc_path))

results_df = pd.concat([d for d in all_results if len(d)], ignore_index=True) if all_results else pd.DataFrame()
display(results_df)

if len(results_df):
    out_path = BASE_DIR / "outputs" / "eval_grid_results.csv"
    out_path.parent.mkdir(parents=True, exist_ok=True)
    results_df.to_csv(out_path, index=False, encoding="utf-8-sig")
    display(results_df.sort_values(["retrieval_mrr@k", "gen_match_rate"], ascending=[False, False]).head(20))
    print("Saved:", out_path)

Embedding: 100%|██████████| 3/3 [00:01<00:00,  2.18it/s]
(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf | page | k=3:   5%|▍         | 1/21 [00:02<00:41,  2.08s/it]

Q: 사업(용역)명은 무엇인가?
GOLD: 벤처확인종합관리시스템 기능 고도화
PRED: NOT_FOUND
----------------------------------------


(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf | page | k=3:  10%|▉         | 2/21 [00:02<00:18,  1.05it/s]

Q: 발주 기관(수요기관)은 어디인가?
GOLD: 벤처기업확인기관
PRED: NOT_FOUND
----------------------------------------


(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf | page | k=3:  14%|█▍        | 3/21 [00:06<00:45,  2.50s/it]

...

Q: 사업 목적(추진 배경)은 무엇인가?
GOLD: ['복수의결권주식', '스톡옵션(주식매수선택권)', '성과조건부주식교부계약(RS) 등의 기능 고도화 및 이관']
PRED: 벤처확인 연계를 통한 복수의결권 및 스톡옵션 시스템 고도화 추진
----------------------------------------


(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf | page_paragraph | k=15: 100%|██████████| 21/21 [00:57<00:00,  2.72s/it]


,doc_id,chunk_strategy,chunk_params,k,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity
0,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page,{},3,21,21,0.666667,0.547619,21,1.0,0.095238,32.330595
1,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page,{},5,21,21,0.761905,0.569048,21,1.0,0.142857,33.771065
2,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page,{},10,21,21,0.857143,0.584921,21,1.0,0.095238,33.691561
3,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page,{},15,21,21,0.904762,0.588322,21,1.0,0.142857,34.674180
4,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,fixed_chars,"{""size"": 1200, ""overlap"": 200}",3,21,21,0.333333,0.333333,21,1.0,0.142857,29.338725
5,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,fixed_chars,"{""size"": 1200, ""overlap"": 200}",5,21,21,0.523810,0.376190,21,1.0,0.142857,30.068149
6,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,fixed_chars,"{""size"": 1200, ""overlap"": 200}",10,21,21,0.761905,0.405423,21,1.0,0.142857,30.613821
7,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,fixed_chars,"{""size"": 1200, ""overlap"": 200}",15,21,21,0.809524,0.408598,21,1.0,0.142857,28.757313
8,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,fixed_chars,"{""size"": 2000, ""overlap"": 300}",3,21,21,0.428571,0.333333,21,1.0,0.047619,4.895517
9,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,fixed_chars,"{""size"": 2000, ""overlap"": 300}",5,21,21,0.666667,0.385714,21,1.0,0.047619,4.895517


,doc_id,chunk_strategy,chunk_params,k,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity
19,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 2500, ""min_chars"": 400}",15,21,21,0.904762,0.596750,21,1.0,0.190476,44.035182
18,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 2500, ""min_chars"": 400}",10,21,21,0.904762,0.596750,21,1.0,0.095238,38.980147
14,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 1500, ""min_chars"": 300}",10,21,21,0.904762,0.596221,21,1.0,0.333333,48.502167
15,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 1500, ""min_chars"": 300}",15,21,21,0.904762,0.596221,21,1.0,0.238095,47.245155
3,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page,{},15,21,21,0.904762,0.588322,21,1.0,0.142857,34.674180
2,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page,{},10,21,21,0.857143,0.584921,21,1.0,0.095238,33.691561
12,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 1500, ""min_chars"": 300}",3,21,21,0.761905,0.579365,21,1.0,0.190476,43.308467
16,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 2500, ""min_chars"": 400}",3,21,21,0.761905,0.579365,21,1.0,0.190476,44.845282
17,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 2500, ""min_chars"": 400}",5,21,21,0.761905,0.579365,21,1.0,0.190476,42.736457
13,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 1500, ""min_chars"": 300}",5,21,21,0.761905,0.579365,21,1.0,0.095238,40.665597


Saved: /home/ohs3201/codeit/codeit-part3-team4/outputs/eval_grid_results.csv


## 11) 최고 조합 찾기

Retrieval/Generation/종합 점수 기준으로 최고 조합을 요약합니다.


In [12]:
if len(results_df):
    best_retrieval = (
        results_df
        .sort_values(["doc_id","retrieval_mrr@k","retrieval_recall@k"], ascending=[True,False,False])
        .groupby("doc_id", as_index=False).head(1)
    )
    display(best_retrieval)

    best_gen = (
        results_df
        .sort_values(["doc_id","gen_match_rate","gen_avg_similarity"], ascending=[True,False,False])
        .groupby("doc_id", as_index=False).head(1)
    )
    display(best_gen)

    df = results_df.copy()
    df["score"] = (
        0.5 * df["retrieval_mrr@k"].fillna(0) +
        0.2 * df["retrieval_recall@k"].fillna(0) +
        0.3 * df["gen_match_rate"].fillna(0)
    )

    best_all = (
        df
        .sort_values(["doc_id","score"], ascending=[True,False])
        .groupby("doc_id", as_index=False).head(1)
    )

    display(best_all[[
        "doc_id","chunk_strategy","chunk_params","k","score",
        "retrieval_mrr@k","retrieval_recall@k","gen_match_rate","gen_avg_similarity"
    ]])
else:
    print("No results to summarize.")

,doc_id,chunk_strategy,chunk_params,k,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity
18,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 2500, ""min_chars"": 400}",10,21,21,0.904762,0.59675,21,1.0,0.095238,38.980147


,doc_id,chunk_strategy,chunk_params,k,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity
14,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 1500, ""min_chars"": 300}",10,21,21,0.904762,0.596221,21,1.0,0.333333,48.502167


,doc_id,chunk_strategy,chunk_params,k,score,retrieval_mrr@k,retrieval_recall@k,gen_match_rate,gen_avg_similarity
14,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 1500, ""min_chars"": 300}",10,0.579063,0.596221,0.904762,0.333333,48.502167


## 12) 지표별 Top-1 조합

지표 단위로 최고 조합을 확인합니다.


In [13]:
def top1_by(metric: str):
    df = results_df.dropna(subset=[metric]).sort_values(metric, ascending=False)
    return df.head(1)[
        ["doc_id","chunk_strategy","chunk_params","k",
         "n_questions_total","n_questions_with_evidence",
         "retrieval_recall@k","retrieval_mrr@k",
         "n_questions_with_gold_fields","gen_fill_rate",
         "gen_match_rate","gen_avg_similarity"]
    ]

display(top1_by("retrieval_mrr@k"))
display(top1_by("retrieval_recall@k"))
display(top1_by("gen_match_rate"))
display(top1_by("gen_avg_similarity"))

,doc_id,chunk_strategy,chunk_params,k,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity
18,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 2500, ""min_chars"": 400}",10,21,21,0.904762,0.59675,21,1.0,0.095238,38.980147


,doc_id,chunk_strategy,chunk_params,k,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity
3,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page,{},15,21,21,0.904762,0.588322,21,1.0,0.142857,34.67418


,doc_id,chunk_strategy,chunk_params,k,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity
14,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 1500, ""min_chars"": 300}",10,21,21,0.904762,0.596221,21,1.0,0.333333,48.502167


,doc_id,chunk_strategy,chunk_params,k,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity
14,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 1500, ""min_chars"": 300}",10,21,21,0.904762,0.596221,21,1.0,0.333333,48.502167


## 13) 문서별(doc_id별) Top-1 조합

문서별로 가장 좋은 조합을 확인합니다.


In [14]:
def best_per_doc(metric: str):
    df = results_df.dropna(subset=[metric]).copy()
    idx = df.groupby("doc_id")[metric].idxmax()
    return df.loc[idx].sort_values(metric, ascending=False)

display(best_per_doc("retrieval_mrr@k"))

,doc_id,chunk_strategy,chunk_params,k,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity
18,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 2500, ""min_chars"": 400}",10,21,21,0.904762,0.59675,21,1.0,0.095238,38.980147


## 14) 종합 점수 Top

Retrieval+Generation을 함께 고려한 종합 스코어 Top을 확인합니다.


In [15]:
df = results_df.copy()

for c in ["retrieval_mrr@k","retrieval_recall@k","gen_match_rate","gen_avg_similarity"]:
    df[c] = df[c].astype(float)

df["score"] = (
    0.6 * df["retrieval_mrr@k"].fillna(0) +
    0.2 * df["retrieval_recall@k"].fillna(0) +
    0.2 * df["gen_match_rate"].fillna(0)
)

display(df.sort_values("score", ascending=False).head(10))

,doc_id,chunk_strategy,chunk_params,k,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity,score
14,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 1500, ""min_chars"": 300}",10,21,21,0.904762,0.596221,21,1.0,0.333333,48.502167,0.605351
15,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 1500, ""min_chars"": 300}",15,21,21,0.904762,0.596221,21,1.0,0.238095,47.245155,0.586304
19,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 2500, ""min_chars"": 400}",15,21,21,0.904762,0.596750,21,1.0,0.190476,44.035182,0.577098
3,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page,{},15,21,21,0.904762,0.588322,21,1.0,0.142857,34.674180,0.562517
18,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 2500, ""min_chars"": 400}",10,21,21,0.904762,0.596750,21,1.0,0.095238,38.980147,0.558050
2,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page,{},10,21,21,0.857143,0.584921,21,1.0,0.095238,33.691561,0.541429
12,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 1500, ""min_chars"": 300}",3,21,21,0.761905,0.579365,21,1.0,0.190476,43.308467,0.538095
16,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 2500, ""min_chars"": 400}",3,21,21,0.761905,0.579365,21,1.0,0.190476,44.845282,0.538095
17,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page_paragraph,"{""max_chars"": 2500, ""min_chars"": 400}",5,21,21,0.761905,0.579365,21,1.0,0.190476,42.736457,0.538095
1,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,page,{},5,21,21,0.761905,0.569048,21,1.0,0.142857,33.771065,0.522381


## 15) 디버깅 함수

특정 instance_id의 검색 결과와 컨텍스트를 확인합니다.


In [16]:
def debug_instance(doc_path: Path, instance_id: str, chunk_name: str="page", chunk_params: dict=None, k: int=10):
    chunk_params = chunk_params or {}
    doc_name = doc_path.name
    pages = extract_pages(doc_path)
    chunks = CHUNKERS[chunk_name](pages, doc_name, **chunk_params)
    texts = [c.text for c in chunks]
    embs = embed_texts(texts)
    index_pack = build_index(embs)

    qrow = questions_df[questions_df["instance_id"].astype(str) == str(instance_id)].iloc[0]
    question = str(qrow["question"])

    qv = embed_query(question)
    _, idxs = search(index_pack, qv, top_k=k)
    ctx = build_context(chunks, idxs)

    print("Question:", question)
    print("Context length:", len(ctx))
    print("Context preview:", ctx[:800])

In [20]:
doc_path = DOC_PATHS[0]  # 첫 번째 문서
debug_instance(doc_path, instance_id="G_Q004", chunk_name="page", k=10)

Embedding: 100%|██████████| 3/3 [00:01<00:00,  2.82it/s]


Question: 총 사업 예산(사업비)은 얼마인가?
Context length: 1894
Context preview: [p4~p4 | p4]\n2 사업개요
□ (사 업 명) 2024년 「벤처확인종합관리시스템 기능 고도화」 용역사업
- 복수의결권주식, 스톡옵션, 성과조건부주식교부 기능 구축 및 고도화
□ (사업기간) 계약일로부터 150일
* 1차 오픈 : 2024.07.10. 성과조건부 및 스톡옵션 오픈
** 2차 오픈 : 2024.09월 中 GrandOpen
□ (소요예산) 352,000,000원(부가가치세 포함)
□ (계약방식) 제한경쟁입찰(협상에 의한 계약)에 의한 선정*
* (근거) 「국가를 당사자로 하는 계약에 관한 법률」 시행령 제21조, 제43조
□ (선정 방법) 본 사업은 전문기술이 요구되는 사업으로 기술력이 높은 업체
를 선정하고 협상을 통해 요구사항을 반영할 수 있는 ’협상에 의한 계약’
방식으로 추진
◦ (선정 절차) 제안서 평가는 기술평가(90%)와 가격평가(10%)를 합산하여
종합평가점수로 산출(배정한도 85% 이상, 고득점순으로 협상적격자 결
정)
행정기관 및 공공기관 정보시스템 구축․운영 지침
제18조(평가배점)
행정기관 등의 장은 기술력이 우수한 사업자를 선정하여 정보화사업의 품질을 확보하기 위해
국가계약법 시행령 제43조의2, 지방계약법 시행령 제44조에 따라서 협상에 의한 계약체결 방법
을 우선적으로 적용하고, 기술능력평가 배점한도를 90점으로 한다.
□ (대기업 참여 제한) 「소프트웨어 진흥법」제48조제2항에 따라, 사업금액
20억원 미만 계약은 대기업 참여 제한
- 4 -\n\n[p98~p98 | p98]\n(별지 제4호 서식) 자본금 및 매출액 현황
자본금 및 매출액 현황(최근 3년)
(단
